In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pylab as plt
import numpy as np
import xarray as xr
import cmaps
import matplotlib as mpl
import matplotlib.path as mpath  
from cartopy.util import add_cyclic_point
import sys
sys.path.append("/public/home/songqh/Documents/python_packages")
from sqh_toolbox import plot_figure
from waf import tnwaf
plt.rcParams["font.family"] = "Liberation Sans"
plt.rcParams["font.weight"] = "regular"

In [ ]:
file0 = xr.open_dataset("/public/home/songqh/my_data/ice_cover/HadISST_ice.nc")
file0.coords['longitude'] = np.mod(file0['longitude'], 360)
file0 = file0.reindex({ 'longitude' : np.sort(file0['longitude'])})
sic = file0['sic'].loc[file0.time.dt.year.isin(np.arange(1979,2025,1))
                        & file0.time.dt.month.isin([6,7,8]),0:-90].groupby('time.year').mean('time')
sic_mean = sic.mean('year')
sic_ano = sic - sic_mean
lon1 = file0['longitude']
lat1 = file0['latitude'].loc[0:-90]

In [ ]:
gph = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/gph_200hPa_1979-2024.npy")
uwnd = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/uwnd_200hPa_1979-2024.npy")
vwnd = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/vwnd_200hPa_1979-2024.npy")
u10 = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/u10_1979-2024.npy")
v10 = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/v10_1979-2024.npy")
t2m = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/t2m_1979-2024.npy")
slp = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/slp_1979-2024.npy")
lon = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/lon.npy")
lat = np.load("/public/home/songqh/project/ISM_impact_Antarctic/data/lat.npy")
ds_rws = xr.open_dataset("/public/home/songqh/project/ISM_impact_Antarctic/data/rws_output.nc")
rws =  ds_rws['s1']

In [ ]:
gph_ano = gph - gph.mean(0)
t2m_ano = t2m - t2m.mean(0)
u10_ano = u10 - u10.mean(0)
v10_ano = v10 - v10.mean(0)
slp_ano = slp - slp.mean(0)
rws_ano = rws - rws.mean(0)

In [ ]:
def get_cmap_pr(n_colors: int, newcmap , method: int = None) -> list:
    index = list(range(1, n_colors + 1))
    color_list = [newcmap(i / n_colors) for i in index]
    if method == 1:
        color_list[0] = [1., 1., 1.]  
    elif method == 2:
        mid_index = len(color_list) // 2
        color_list[mid_index] = [1., 1., 1.]  
        color_list[mid_index - 1] = [1., 1., 1.]
    return color_list
new_CBR_coldhot = mpl.colors.ListedColormap(get_cmap_pr(10,cmaps.CBR_coldhot,method=2))
new_cmocean_balance_r = mpl.colors.ListedColormap(get_cmap_pr(20,cmaps.cmocean_balance_r,method=2))

In [ ]:
_,px,py = tnwaf(gph_ano[-2],uwnd,vwnd,lon,lat,200)
px_copy = px.copy()
py_copy = py.copy()
px_copy[:110,:] = np.nan
py_copy[:110,:] = np.nan

In [ ]:
cycle_gph_ano,cycle_lon = add_cyclic_point(gph_ano, coord=lon)
cycle_rws_ano,cycle_lon = add_cyclic_point(rws_ano, coord=lon)

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(111,projection = ccrs.PlateCarree(central_longitude=150))
ax1.draw_map([-60,280,-90,30],latspec=30) 
ax1.set_title("GPH&RWS&WAF@200hPa",fontsize=20,loc='center')
ax1.set_title("c",fontsize=24,weight='bold',loc='left')
cf1 = ax1.contourf(cycle_lon,lat,cycle_gph_ano[-2]/9.8, zorder=2,
        levels=np.linspace(-200,200,11),extend = 'both',
        transform=ccrs.PlateCarree(), cmap=new_CBR_coldhot)
cycle_rws_ano[:,70:110,:] = 0
cycle_rws_ano[:,160:180,:] = 0
ax1.contour(cycle_lon,lat, cycle_rws_ano[-2],colors='purple', zorder=3,levels=[2,4,6,8,10,12],lw=3,transform=ccrs.PlateCarree())
ax1.contour(cycle_lon,lat, cycle_rws_ano[-2],colors='purple', zorder=3,levels=[-4,-2],lw=3,transform=ccrs.PlateCarree())
kk = 7
q = ax1.quiver(lon[::kk],lat[::kk],px_copy[::kk,::kk],py_copy[::kk,::kk],alpha=0.7,color='g',scale=50, zorder=100,
               width=0.003,headwidth=4, headlength=5, headaxislength=4,transform=ccrs.PlateCarree())
qk = ax1.quiverkey(q, 1.04, 0.99, U=3, label='3',labelpos='N',
    zorder=3, coordinates='axes',color='g',fontproperties={'size': 18},
    labelcolor='k')
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.35,pad=0.02)
cb.ax.set_colorbar(minor='off')
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctic/figures/response_figures/support-8c.jpeg",dpi=600, bbox_inches='tight')

In [ ]:
cycle_data,cycle_lon = add_cyclic_point(slp_ano, coord=lon)
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(111,projection = ccrs.PlateCarree(central_longitude=150))
ax1.draw_map([-60,280,-90,30],latspec=30)
ax1.set_title("SLP&wind10m",fontsize=20,loc='center')
ax1.set_title("d",fontsize=24,weight='bold',loc='left')
cf1 = ax1.contourf(cycle_lon,lat,cycle_data[-2], zorder=2,levels=np.arange(-1200,1300,100),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=new_CBR_coldhot)
kk=7
u10_ano[-2,174:182,:],v10_ano[-2,174:182,:] = np.nan, np.nan
q = ax1.quiver(lon[::kk],lat[::kk],u10_ano[-2,::kk,::kk],v10_ano[-2,::kk,::kk],color='k',
           alpha=0.9,zorder=3,scale=50,headwidth=3,width=0.003,transform=ccrs.PlateCarree())
qk = ax1.quiverkey(q, 1.04, 0.99, U=3, label='3',labelpos='N',
    zorder=3, coordinates='axes',color='k',fontproperties={'size': 18},
    labelcolor='k')
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.35,pad=0.02)
cb.ax.set_colorbar(minor='off')
cb.set_ticks(np.arange(-1200,1800,600))
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctic/figures/response_figures/support-8d.jpeg",dpi=600, bbox_inches='tight')


In [ ]:
cycle_data,cycle_lon = add_cyclic_point(t2m_ano, coord=lon)
leftlon, rightlon, lowerlat, upperlat = (-180,180,-50,-90)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
fig = plt.figure(figsize=(5,4))
ax1 = plt.axes(projection=ccrs.Orthographic(central_longitude=120.0, central_latitude=-90.0, globe=None))
ax1.gridlines(linestyle='--',zorder=2)
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax1.set_boundary(circle, transform=ax1.transAxes)
ax1.add_feature(cfeature.COASTLINE.with_scale('110m'),color='k',linewidth=1.5,zorder=3)
ax1.coastlines(resolution='110m', color='black', linewidth=1,zorder=3)
ax1.set_title("SAT&wind10m",fontsize=20,loc='center')
ax1.set_title("a",fontsize=24,weight='bold',loc='left')
ax1.set_extent(img_extent, ccrs.PlateCarree())
kk=5
u10_ano[-2,174:182,:],v10_ano[-2,174:182,:] = np.nan, np.nan
q = ax1.quiver(lon[::kk],lat[::kk],u10_ano[-2,::kk,::kk],v10_ano[-2,::kk,::kk],color='k',
           alpha=0.9,zorder=3,scale=30,headwidth=5,width=0.004,transform=ccrs.PlateCarree())
qk = ax1.quiverkey(q, 1.08, 0.89, U=3, label='3',labelpos='N',
    zorder=3, coordinates='axes',color='k',fontproperties={'size':18},
    labelcolor='k')
cf1 = ax1.contourf(cycle_lon,lat,cycle_data[-2], zorder=2,levels=np.arange(-7,8,1),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=cmaps.BlueWhiteOrangeRed)
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.7,pad=0.035)
cb.ax.set_colorbar(minor='off')

fig.savefig("/public/home/songqh/project/ISM_impact_Antarctic/figures/response_figures/support-8a.jpeg",dpi=600, bbox_inches='tight')


In [ ]:
cycle_data,cycle_lon = add_cyclic_point(sic_ano, coord=lon1)
leftlon, rightlon, lowerlat, upperlat = (-180,180,-50,-90)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
fig = plt.figure(figsize=(5,4))
ax1 = plt.axes(projection=ccrs.Orthographic(central_longitude=120.0, central_latitude=-90.0, globe=None))
ax1.gridlines(linestyle='--',zorder=2)
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
ax1.set_boundary(circle, transform=ax1.transAxes)
ax1.add_feature(cfeature.COASTLINE.with_scale('110m'),color='k',linewidth=1.5,zorder=3)
ax1.add_feature(cfeature.LAND, facecolor='lightgray',zorder=3)
ax1.coastlines(resolution='110m', color='black', linewidth=1,zorder=3)
ax1.set_title("SIC",fontsize=20,loc='center')
ax1.set_title("b",fontsize=24,weight='bold',loc='left')
ax1.set_extent(img_extent, ccrs.PlateCarree())
cf1 = ax1.contourf(cycle_lon,lat1,cycle_data[-2], zorder=2,levels=np.linspace(-0.5,0.5,21),  
                   extend = 'both',transform=ccrs.PlateCarree(), cmap=new_cmocean_balance_r)
cb = fig.colorbar(cf1,orientation='vertical',shrink=0.7,pad=0.035)
cb.ax.set_colorbar(minor='off')
fig.savefig("/public/home/songqh/project/ISM_impact_Antarctic/figures/response_figures/support-8b.jpeg",dpi=600, bbox_inches='tight')